#  Adam 算法
**集大成者**

Adam虽然是DL中使用的更强大和有效的优化算法之一，它也有一些缺点：Adam算法可能由于方差控制不良而发散，18年的时候有人给Adam算法提供了一个称为Yogi的热补丁，解决了以上的问题。

## 具体公式
Adam速算法使用指数加权移动平均值来估算梯度的动量和二次矩，即使用状态变量：
$$
v_t \gets \beta_1 v_{t-1} + (1-\beta_1)g_t,\\
s_t \gets \beta_2 s_{t-1} + (1-\beta_2) g_t^2
$$
一般取$$\beta_1 = 0.9, \beta_2 = 0.999$$，方差估计的移动远远比动量估计移动的慢。同样地，我们需要使用:
$$
\sum_{t=0}^t \beta^i = \frac{1-\beta^t}{1- \beta}
$$
来对$v_t,s_t$来进行归一化：
$$
\hat v_t = \frac {v_t}{1-\beta_1^t}\\
\hat s_t = \frac {s_t}{1-\beta_2^t}
$$

有了正确的估计$\hat v_t ,\hat s_t$，可以使用非常类似于RMSProp算法的方式重新缩放梯度：
$$
g_t^\prime = \frac{ \eta \hat v_t}{\sqrt{\hat s_t}+ \epsilon}
$$

与RMSProp不同的是：使用$\hat v_t$来更新梯度而不是梯度$g_t$本身，Adam使用${\sqrt{\hat s_t}+ \epsilon}$而不是${\sqrt{\hat s_t+ \epsilon}}$进行缩放。

最后的最后，使用平滑之后的梯度对参数进行更新：
$$x_t \gets x_{t-1} - g_t^\prime$$


In [ ]:
import torch as t

def init_state(feature_dim):
    v_w,v_b = t.zeros((feature_dim,1)),t.zeros(1)
    s_w,s_b = t.zeros((feature_dim,1)),t.zeros(1)
    return((v_w,s_w),(v_b,s_b))

def adam(params:t.Tensor,states,hyperparams):
    beta1 ,beta2 ,eps = 0.9,0.999,1e-6
    for p,(v,s) in zip(params,states):
        with t.no_grad():
            v[:] = beta1 * v + (1-beta1) * p.grad
            s[:] = beta2 * s + (1-beta2) * t.square(p.grad)
            v_bias_corr = v/(1-beta1 ** hyperparams["t"])
            s_bias_corr = s/(1-beta2**hyperparams["t"])
            p[:] -= hyperparams["lr"] * v_bias_corr /(t.sqrt(s_bias_corr)+eps)
        p.grad.data.zero_()

    hyperparams["t"] += 1